In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split
import os

# Configuration
path = "Dataset/Dataset"  # Updated path to the inner 'Dataset' folder
batch_size_val = 32
epochs_val = 10
imageDimensions = (32, 32, 3)
testRatio = 0.2
validationRatio = 0.2

count = 0
images = []
classNo = []
myList = os.listdir(path)
print("Total Classes Detected:", len(myList))
noOfClasses = len(myList)
print("Importing Classes.....")

for className in myList:
    classPath = os.path.join(path, className)
    if os.path.isdir(classPath):  # Check if it's a directory
        myPicList = os.listdir(classPath)
        for y in myPicList:
            curImg = cv2.imread(os.path.join(classPath, y))
            if curImg is not None:  # Ensure image is read correctly
                images.append(curImg)
                classNo.append(count)
        print(f"Class {className} imported with {len(myPicList)} images.")
        count += 1

print(" ")
images = np.array(images)
classNo = np.array(classNo)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio)

# Data preprocessing
print("Data Shapes")
print("Train", end=" "); print(X_train.shape, y_train.shape)
print("Validation", end=" "); print(X_validation.shape, y_validation.shape)
print("Test", end=" "); print(X_test.shape, y_test.shape)

def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def preprocessing(img):
    img = grayscale(img)     
    img = equalize(img)      
    img = img / 255            
    return img

X_train = np.array(list(map(preprocessing, X_train)))  
X_validation = np.array(list(map(preprocessing, X_validation)))
X_test = np.array(list(map(preprocessing, X_test)))

# Reshape the data
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataGen = ImageDataGenerator(width_shift_range=0.1,   
                             height_shift_range=0.1,
                             zoom_range=0.2,  
                             shear_range=0.1,  
                             rotation_range=10)  
dataGen.fit(X_train)

# Model creation
y_train = to_categorical(y_train, noOfClasses)
y_validation = to_categorical(y_validation, noOfClasses)
y_test = to_categorical(y_test, noOfClasses)

def myModel():
    model = Sequential()
    model.add(Conv2D(60, (5, 5), input_shape=(imageDimensions[0], imageDimensions[1], 1), activation='relu'))
    model.add(Conv2D(60, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses, activation='softmax')) 
    model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = myModel()
print(model.summary())

# Fit the model
history = model.fit(dataGen.flow(X_train, y_train, batch_size=batch_size_val),
                    steps_per_epoch=len(X_train) // batch_size_val,
                    epochs=epochs_val,
                    validation_data=(X_validation, y_validation),
                    shuffle=1)

# Plotting results
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('Epoch')

plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'validation'])
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.show()

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Score:', score[0])
print('Test Accuracy:', score[1])

# Save the model
model.save("model.h5")


Total Classes Detected: 43
Importing Classes.....
Class 0 imported with 180 images.
Class 1 imported with 1980 images.
Class 10 imported with 1800 images.
Class 11 imported with 1170 images.
Class 12 imported with 1890 images.
Class 13 imported with 1920 images.
Class 14 imported with 690 images.
Class 15 imported with 540 images.
Class 16 imported with 360 images.
Class 17 imported with 990 images.
Class 18 imported with 1080 images.
Class 19 imported with 180 images.
Class 2 imported with 2010 images.
Class 20 imported with 300 images.
Class 21 imported with 270 images.
Class 22 imported with 330 images.
Class 23 imported with 450 images.
Class 24 imported with 240 images.
Class 25 imported with 1350 images.
Class 26 imported with 540 images.
Class 27 imported with 210 images.
Class 28 imported with 480 images.
Class 29 imported with 240 images.
Class 3 imported with 1260 images.
Class 30 imported with 390 images.
Class 31 imported with 690 images.
Class 32 imported with 210 images.


C:\Users\tanma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 60)          │           1,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 60)          │          90,060 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 12, 12, 60)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 10, 10, 30)          │          16,230 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 30)            │           8,130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 4, 4, 30)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4, 4, 30)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 480)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 500)                 │         240,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 43)                  │          21,543 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 378,023 (1.44 MB)

...

None
Epoch 1/10


C:\Users\tanma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


695/695 ━━━━━━━━━━━━━━━━━━━━ 153s 212ms/step - accuracy: 0.1634 - loss: 3.0961 - val_accuracy: 0.7773 - val_loss: 0.8347
Epoch 2/10
  1/695 ━━━━━━━━━━━━━━━━━━━━ 2:20 202ms/step - accuracy: 0.4688 - loss: 1.7725

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


695/695 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.4688 - loss: 1.7725 - val_accuracy: 0.7804 - val_loss: 0.8278
Epoch 3/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 147s 212ms/step - accuracy: 0.5877 - loss: 1.3340 - val_accuracy: 0.9028 - val_loss: 0.3430
Epoch 4/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.7812 - loss: 0.6322 - val_accuracy: 0.8849 - val_loss: 0.3745
Epoch 5/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 147s 211ms/step - accuracy: 0.7363 - loss: 0.8533 - val_accuracy: 0.9567 - val_loss: 0.1841
Epoch 6/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.7812 - loss: 0.5653 - val_accuracy: 0.9599 - val_loss: 0.1752
Epoch 7/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 150s 216ms/step - accuracy: 0.7969 - loss: 0.6439 - val_accuracy: 0.9582 - val_loss: 0.1381
Epoch 8/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.8125 - loss: 0.6245 - val_accuracy: 0.9560 - val_loss: 0.1420
Epoch 9/10
324/695 ━━━━━━━━━━━━━━━━━━━━ 1:19 215ms/step - accuracy: 0.8243 - loss: 0.5501